In [1]:
import pandas as pd
import sys
sys.path.append("..")

In [32]:
from scipy.stats import wilcoxon

def eval_wilcoxon(a, b, alpha = 0.05/(30), a_name=None, b_name=None, verbose=True):
    stat, p = wilcoxon(a, b)
    if verbose:
        print(len(a), len(b))
        print(p)
        if p > alpha:
            print(f'Same distribution (fail to reject H0) btw {a_name} & {b_name}')
        else:
            print('Different distribution (reject H0)')
            print(f"\t{a_name}: {a.mean():,.3} vs. {b_name}: {b.mean():,.3}")
    else:
        return(p, p>alpha)

In [33]:
cat = pd.read_excel("../data/categorical.xlsx", header=1, index_col=0)
con = pd.read_excel("../data/continuous.xlsx", header=1, index_col=0)
mix = pd.read_excel("../data/mixed.xlsx", header=1, index_col=0)

cat.dropna(inplace=True)
con.dropna(inplace=True)
mix.dropna(inplace=True)

for d in [cat, con, mix]:
    print(d.shape)

df = pd.concat([cat, con, mix])
df.shape

(35, 10)
(42, 10)
(35, 10)


(112, 10)

In [34]:
df.head(3)

,G-mean,F1,AUC,G-mean.1,F1.1,AUC.1,G-mean.2,F1.2,AUC.2,data_name
ROSE,0.574377,0.093301,0.628403,0.006358,0.068332,0.500101,0.000000,0.068320,0.500000,boundary.csv
SMOTE,0.127559,0.038775,0.509986,0.005080,0.001270,0.500326,0.238614,0.071886,0.525882,boundary.csv
ADASYN,0.126475,0.038034,0.509735,0.003810,0.000952,0.500242,0.233110,0.071777,0.525227,boundary.csv


In [75]:
p_dict = {}
same_dict = {}
for metric in df.columns[:-1]:
    print(metric)
    p_li = []
    same_li = []
    for m in ['ROSE', 'SMOTE', 'ADASYN', 'MWMOTE', 'ROS']:
        p, s = eval_wilcoxon(df.loc[m, metric], df.loc['RNRvs', metric], a_name = m, b_name = 'RNR', verbose=False)
        p_li.append(p)
        same_li.append(s)
    p_dict[metric] = p_li
    same_dict[metric] = same_li

G-mean
F1
AUC
G-mean.1
F1.1
AUC.1
G-mean.2
F1.2
AUC.2


In [76]:
p_res = pd.DataFrame(p_dict, index=['ROSE', 'SMOTE', 'ADASYN', 'MWMOTE', 'ROS'])
same_res = pd.DataFrame(same_dict, index=['ROSE', 'SMOTE', 'ADASYN', 'MWMOTE', 'ROS'])

display(p_res)
display(same_res)

,G-mean,F1,AUC,G-mean.1,F1.1,AUC.1,G-mean.2,F1.2,AUC.2
ROSE,0.000214,0.065399,0.000153,0.000031,0.005157,0.000061,0.000031,0.000763,0.000031
SMOTE,0.000580,0.192810,0.001007,0.000061,0.065399,0.000153,0.002686,0.028992,0.004181
ADASYN,0.000763,0.495422,0.000763,0.000061,0.083252,0.000092,0.001312,0.013092,0.001007
MWMOTE,0.000031,0.596588,0.000031,0.000061,0.375458,0.000061,0.000031,0.003357,0.000031
ROS,0.000580,0.192810,0.001007,0.000031,0.433197,0.000031,0.000214,0.003357,0.001007


,G-mean,F1,AUC,G-mean.1,F1.1,AUC.1,G-mean.2,F1.2,AUC.2
ROSE,False,True,False,False,True,False,False,False,False
SMOTE,False,True,False,False,True,False,True,True,True
ADASYN,False,True,False,False,True,False,False,True,False
MWMOTE,False,True,False,False,True,False,False,True,False
ROS,False,True,False,False,True,False,False,True,False


In [78]:
for m in ['ROSE', 'SMOTE', 'ADASYN', 'MWMOTE', 'ROS']:
    print(m)
    for x in p_res.loc[m].values:
        print(str(round(x, 6))+" &")

ROSE
0.000214 &
0.065399 &
0.000153 &
3.1e-05 &
0.005157 &
6.1e-05 &
3.1e-05 &
0.000763 &
3.1e-05 &
SMOTE
0.00058 &
0.19281 &
0.001007 &
6.1e-05 &
0.065399 &
0.000153 &
0.002686 &
0.028992 &
0.004181 &
ADASYN
0.000763 &
0.495422 &
0.000763 &
6.1e-05 &
0.083252 &
9.2e-05 &
0.001312 &
0.013092 &
0.001007 &
MWMOTE
3.1e-05 &
0.596588 &
3.1e-05 &
6.1e-05 &
0.375458 &
6.1e-05 &
3.1e-05 &
0.003357 &
3.1e-05 &
ROS
0.00058 &
0.19281 &
0.001007 &
3.1e-05 &
0.433197 &
3.1e-05 &
0.000214 &
0.003357 &
0.001007 &


In [54]:
OREM = pd.read_csv("../data/RES_Willcoxn.csv")
#OREM.insert(2, "svmAUC", OREM_AUC.iloc[:, 0])

,svmGmean,svmF1,RFGmean,RFF1,NBGmean,NBF1
0,0.813998,0.182877,0.818466,0.206180,0.727178,0.138629
1,0.177957,0.079568,0.805754,0.578956,0.709696,0.504167
2,0.899168,0.521983,0.887282,0.372922,0.920978,0.223907
3,0.358069,0.114169,0.532564,0.258521,0.710411,0.278259
4,0.637815,0.105526,0.305419,0.126483,0.337699,0.120909
5,0.695191,0.555145,0.458587,0.345898,0.783122,0.409046
6,0.876793,0.748915,0.926143,0.650412,0.881152,0.643241
7,0.005552,0.000647,0.507516,0.217628,0.642112,0.183783
8,0.011229,0.002837,0.240942,0.070664,0.340386,0.083473
9,0.323802,0.172411,0.605324,0.176471,0.717462,0.239468


In [83]:
ours = "RNRvs"
for idx in range(len(OREM.columns)):
    orem = OREM[OREM.columns[idx]]
    rnr = df.loc[ours].iloc[:, idx]
    
    print(f"\n{OREM.columns[idx]}")
    eval_wilcoxon(orem, rnr, a_name = "OREM", b_name = ours, alpha = 0.05/(6*5), verbose=True)
    #print(str(round(a, 6))+" &")


svmGmean
16 16
0.083251953125
Same distribution (fail to reject H0) btw OREM & RNRvs

svmF1
16 16
0.860260009765625
Same distribution (fail to reject H0) btw OREM & RNRvs

svmAUC
16 16
0.00762939453125
Same distribution (fail to reject H0) btw OREM & RNRvs

RFGmean
16 16
0.375457763671875
Same distribution (fail to reject H0) btw OREM & RNRvs

RFF1
16 16
0.781951904296875
Same distribution (fail to reject H0) btw OREM & RNRvs

RFAUC
16 16
0.00042724609375
Different distribution (reject H0)
	OREM: 0.749 vs. RNRvs: 0.805

NBGmean
16 16
0.820892333984375
Same distribution (fail to reject H0) btw OREM & RNRvs

NBF1
16 16
0.528167724609375
Same distribution (fail to reject H0) btw OREM & RNRvs

NBAUC
16 16
0.02496337890625
Same distribution (fail to reject H0) btw OREM & RNRvs


In [7]:
ours = "RNR"
for idx, metric in enumerate([c for c in df.columns[:-1] if "AUC" not in c]):
    print(df.loc[ours, metric].shape, metric)

(16,) G-mean
(16,) F1
(16,) G-mean.1
(16,) F1.1
(16,) G-mean.2
(16,) F1.2


In [ ]:
ours = "RNR"
#ours = "RNRvs"
for idx, metric in enumerate([c for c in df.columns[:-1] if "AUC" not in c]):
    rnr = df.loc[ours, metric]
    orem = OREM[OREM.columns[idx]]
    print(f"\n{OREM.columns[idx]}")
    eval_wilcoxon(orem, rnr, a_name = "OREM", b_name = ours, alpha = 0.05/(6*5))

In [24]:
ours = "RNR"
#ours = "RNRvs"
for idx, metric in enumerate([c for c in df.columns[:-1] if "AUC" not in c]):
    rnr = df.loc[ours, metric]
    orem = OREM[OREM.columns[idx]]
    print(f"\n{OREM.columns[idx]}")
    eval_wilcoxon(orem, rnr, a_name = "OREM", b_name = ours, alpha = 0.05/(6*5))


svmGmean
16 16
0.129730224609375
Same distribution (fail to reject H0)

svmF1
16 16
0.860260009765625
Same distribution (fail to reject H0)

RFGmean
16 16
0.375457763671875
Same distribution (fail to reject H0)

RFF1
16 16
0.820892333984375
Same distribution (fail to reject H0)

NBGmean
16 16
0.820892333984375
Same distribution (fail to reject H0)

NBF1
16 16
0.596588134765625
Same distribution (fail to reject H0)


In [77]:
df.columns[:-1]

Index(['G-mean', 'F1', 'AUC', 'G-mean.1', 'F1.1', 'AUC.1', 'G-mean.2', 'F1.2',
       'AUC.2'],
      dtype='object')

In [87]:
classifier = ["G-mean", "F1", "AUC"]
classifier = [c+"" for c in classifier]
for m in ['ROSE', 'SMOTE', 'ADASYN', 'MWMOTE', 'ROS']:
    eval_wilcoxon(
        df.loc[m, classifier].unstack(),
        df.loc['RNRvs', classifier].unstack(),
        a_name = m, b_name = 'RNR'
    )

48 48
Different distribution (reject H0)
	ROSE: 0.6 vs. RNR: 0.661
48 48
Different distribution (reject H0)
	SMOTE: 0.599 vs. RNR: 0.661
48 48
Different distribution (reject H0)
	ADASYN: 0.593 vs. RNR: 0.661
48 48
Different distribution (reject H0)
	MWMOTE: 0.542 vs. RNR: 0.661
48 48
Different distribution (reject H0)
	ROS: 0.604 vs. RNR: 0.661


In [94]:
metrics = ["G-mean"] * 3
classif = ["", ".1", ".2"]
metrics = [m+classif[idx] for idx, m in enumerate(metrics)]

for m in ['ROSE', 'SMOTE', 'ADASYN', 'MWMOTE', 'ROS']:
    eval_wilcoxon(
        df.loc[m, metrics].unstack(),
        df.loc['RNR', metrics].unstack(),
        a_name = m, b_name = 'RNR'
    )

48 48
Different distribution (reject H0)
	ROSE: 0.608 vs. RNR: 0.776
48 48
Different distribution (reject H0)
	SMOTE: 0.618 vs. RNR: 0.776
48 48
Different distribution (reject H0)
	ADASYN: 0.613 vs. RNR: 0.776
48 48
Different distribution (reject H0)
	MWMOTE: 0.584 vs. RNR: 0.776
48 48
Different distribution (reject H0)
	ROS: 0.606 vs. RNR: 0.776


In [102]:
for m in ['ROSE', 'SMOTE', 'ADASYN', 'MWMOTE', 'ROS']:
    eval_wilcoxon(
        df.loc[m, df.columns[:-1]].unstack(),
        df.loc['RNRvs', df.columns[:-1]].unstack(),
        a_name = m, b_name = 'RNR'
    )

144 144
7.79730502868465e-20
Different distribution (reject H0)
	ROSE: 0.522 vs. RNR: 0.638
144 144
1.2000293116507564e-08
Different distribution (reject H0)
	SMOTE: 0.558 vs. RNR: 0.638
144 144
2.3291873807191104e-10
Different distribution (reject H0)
	ADASYN: 0.55 vs. RNR: 0.638
144 144
5.131079965456826e-15
Different distribution (reject H0)
	MWMOTE: 0.522 vs. RNR: 0.638
144 144
2.0719242382956793e-11
Different distribution (reject H0)
	ROS: 0.548 vs. RNR: 0.638
